In [1]:
# -*- coding: utf-8 -*-
# Python 3.12.6

# Internal Libs
import random
import os
from datetime import datetime
now = datetime.now()

# External Libs
import numpy as np
import pandas as pd

# Produced Libs:
## Data loader
from scripts.data_loader import DataLoader

## Filters
### Filters for AIS data
from scripts.filters.ais_filters import AISFilters as af, AISInterpolation as ai
### Filters for Norsat data
from scripts.filters.norsat_filters import NorsatFilters as nf
### Filters for SAR data
from scripts.filters.sar_filters import SARFilters as sf

## Matching algorithms
### Nearest neighbour
from scripts.matchers.nearest_neighbour_matching import *

#
#from scripts.data_formatter import DataProcessor as dp
#from scripts.matching import hungarian_method
#from scripts.plotter import Plotter as pl

In [2]:
# Define date and time filter
date_key = '03-11-2022'

# PATHS, dataframe and shpfile #
# Define paths
base_path = "C:\\Users\\abelt\\OneDrive\\Desktop\\Kandidat\\"
## File names ##
# AIS
ais_files = {
    '02-11-2022': 'ais\\ais_110215.csv',
    '03-11-2022': 'ais\\ais_110315.csv',
    '05-11-2022': 'ais\\ais_1105.csv'
}
# SAR
sar_files = {
    '02-11-2022': 'sar\\Sentinel_1_detection_20221102T1519.json',
    '03-11-2022': 'sar\\Sentinel_1_detection_20221103T154515.json',
    '05-11-2022': 'sar\\Sentinel_1_detection_20221105T162459.json'
}
# Norsat
norsat_files = {
    '02-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-02T151459Z.json',
    '03-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-03T152759Z.json',
    '05-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-05T155259Z.json'
}

In [3]:
# LOADING #
data_loader = DataLoader(base_path = base_path, ais_files = ais_files, sar_files = sar_files, norsat_files = norsat_files, date_key = date_key)
ais_loader, sar_loader, norsat_loader = data_loader.load_data()

######### SAR #########
# images by date_key: sar_loader.dfs_sar
# objects by date_key: sar_loader.sar_object_dfs
######### AIS #########
#ais_loader.dfs_ais
######### Norsat #########
#norsat_loader.dfs_norsat

sar_data = sar_loader.sar_object_dfs[date_key].copy()
ais_data = ais_loader.dfs_ais[date_key].copy()
norsat_data = norsat_loader.dfs_norsat[date_key].copy()

AIS Data Loaded:
dict_keys(['02-11-2022', '03-11-2022', '05-11-2022'])
Columns for the first DataFrame: Index(['ais_id', 'mmsi', 'TimeStamp', 'latitude', 'longitude', 'length',
       'width', 'sog', 'cog', 'source'],
      dtype='object')
Norsat Data Loaded:
dict_keys(['02-11-2022', '03-11-2022', '05-11-2022'])
Columns for the first DataFrame: Index(['norsat_id', 'TimeStamp', 'latitude', 'longitude',
       'CollectionInformation', 'NRDEmitterPosition', 'CandidateList',
       'source', 'UncertaintyEllipsePoints'],
      dtype='object')
SAR Data Loaded:
dict_keys(['02-11-2022', '03-11-2022', '05-11-2022'])
Columns for the first DataFrame: Index(['ProductType', 'Polarization', 'Swath', 'TimeStamp', 'TimeStamp_end',
       'Name', 'Satellite', 'Shape', 'Objects', 'source'],
      dtype='object')
SAR object Loaded:
dict_keys(['02-11-2022', '03-11-2022', '05-11-2022'])
Columns for the first DataFrame: Index(['sar_id', 'sar_image_id', 'TimeStamp', 'latitude', 'longitude', 'width',
       '

In [4]:
# Filter SAR data
## Filter out SAR data based on on_sea
sar_data = sar_data[sar_data['on_sea'] == True] # This is the SAR data that is on sea
## Filter out SAR data based on class 0
filtered_sar_data = sf.filter_sar_probabilities(sar_data) # This is the SAR data that is class 0

# Filter AIS data
## Define the time threshold for comparison
delta_time = pd.Timedelta(hours=1)
## Finding vessels in AIS data that are close to objects in SAR and Norsat
mmsi_ais_sar = af.ais_find_matching_vessels(AIS_data=ais_data, comparison_data=sar_data, delta_time=delta_time) # This is the AIS data that is close to SAR data time
mmsi_ais_norsat = af.ais_find_matching_vessels(AIS_data=ais_data, comparison_data=norsat_data, delta_time=delta_time) # This is the AIS data that is close to Norsat data time
## Filter AIS data by MMSI
filtered_ais_to_sar = af.filter_by_mmsi(mmsi_ais_sar, ais_data) # This is the AIS data by MMSI numbers that are close to SAR data
filtered_ais_to_norsat = af.filter_by_mmsi(mmsi_ais_norsat, ais_data) # This is the AIS data by MMSI numbers that are close to Norsat data
len(filtered_ais_to_sar.groupby('mmsi')), len(filtered_ais_to_norsat.groupby('mmsi'))

(2515, 2520)

In [5]:
filtered_ais_to_norsat

,ais_id,mmsi,TimeStamp,latitude,longitude,length,width,sog,cog,source
3,3,992576146,2022-11-03 14:02:28+00:00,68.601290,15.349760,2.0,2.0,NaN,NaN,ais
4,4,992576146,2022-11-03 14:08:56+00:00,68.601290,15.349760,2.0,2.0,NaN,NaN,ais
5,5,992576146,2022-11-03 14:15:25+00:00,68.601290,15.349760,2.0,2.0,NaN,NaN,ais
6,6,992576146,2022-11-03 14:21:53+00:00,68.601290,15.349760,2.0,2.0,NaN,NaN,ais
7,7,992576146,2022-11-03 14:28:21+00:00,68.601290,15.349760,2.0,2.0,NaN,NaN,ais
...,...,...,...,...,...,...,...,...,...,...
75869,75869,538009667,2022-11-03 16:18:57+00:00,69.595167,33.544667,180.0,30.0,10.7,181.0,ais
75870,75870,538009667,2022-11-03 17:04:28+00:00,69.457000,33.553333,180.0,30.0,11.6,180.0,ais
75871,75871,538009667,2022-11-03 17:10:24+00:00,69.437833,33.552333,180.0,30.0,11.3,181.0,ais
75872,75872,538009667,2022-11-03 17:20:56+00:00,69.404667,33.551667,180.0,30.0,11.4,181.0,ais


In [6]:
# Interpolate MMSI points for filtered AIS data to SAR and Norsat
## Interpolate MMSI points for filtered AIS data to SAR
interpolated_ais_sar, not_enough_sar = ai.ais_interpolate_mmsi_points(filtered_ais_to_sar, sar_data['TimeStamp'])
## Interpolate MMSI points for filtered AIS data to Norsat
interpolated_ais_norsat, not_enough_norsat = ai.ais_interpolate_mmsi_points(filtered_ais_to_norsat, norsat_data['TimeStamp'])

MMSI numbers with insufficient data points: 99 out of 2515
MMSI numbers with insufficient data points: 98 out of 2520


In [13]:
interpolated_ais_sar['sog_after'].isna().value_counts()#.sum(), interpolated_ais_norsat['sog_after'].isna().sum()

sog_after
False    2340
True       76
Name: count, dtype: int64

0       0.0
1       0.5
2       0.1
3       0.0
4      -0.1
       ... 
2411    NaN
2412    NaN
2413    NaN
2414    NaN
2415    NaN
Length: 2416, dtype: float64

In [ ]:
d = mmsi_ais_sar.get_group(209610000)
d

In [ ]:
filtered_ais_to_sar

In [ ]:
interpolated_ais_norsat

```markdown
### List of MMSI Numbers within Delta Time in `matching_vessels_ais_sar`

To get a list of all MMSI numbers that are within the `delta_time` in `matching_vessels_ais_sar`, we can extract the keys from the grouped DataFrame. Here's the code to achieve this:

```python
mmsi_list = list(matching_vessels_ais_sar.groups.keys())
print(mmsi_list)
```

This will provide a list of all MMSI numbers that have matching timestamps within the specified `delta_time`.
```

In [19]:
#print('Interpolation for ais')
#print('SAR')
#ais_sar_interpol, sar_missing = dp.ais_interpolate_mmsi_points(ais_data = ais_data['tm_ais_sar'], date_key = date_key, interpolation_time_col = sar_data['sar_landmasked']['TimeStamp'])
#print('Norsat-3')
#ais_norsat_interpol, norsat_missing = dp.ais_interpolate_mmsi_points(ais_data = ais_data['tm_ais_norsat'], date_key = date_key, interpolation_time_col = norsat_data[date_key]['TimeStamp'])

In [20]:
#time_matching_ais_sar = ma.ais_find_matching_vessels(AIS_data., SAR_data, self.date_key, time_diff)

# Perform time matching between AIS and Norsat if both are available
#time_matching_ais_norsat = ma.ais_find_matching_vessels(AIS_data, norsat_data, self.date_key, time_diff)

# Prepare the data dictionaries for output
#ais_data = {'ais': AIS_data, 'tm_ais_sar': time_matching_ais_sar, 'tm_ais_norsat': time_matching_ais_norsat}
#sar_data = {'sar': SAR_data, 'sar_objects': SAR_expanded, 'sar_landmasked': SAR_on_sea}


In [ ]:

matched_data3 = match_nearest_unique(norsat_data, sar_data, 'norsat_id', 'sar_id')
#matched_data3